## Deliverable 2. Create a Customer Travel Destinations Map.

In [9]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import csv
import os
# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [10]:
# 1. Import the WeatherPy_database.csv file. 
file_to_load = os.path.join('..', 'Weather_Database', 'WeatherPy_database.csv')
city_data_df = pd.read_csv(file_to_load)
city_data_df.head()
 



,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,0,Bethel,US,41.3712,-73.4140,30.99,79,8,9.19,clear sky
1,1,Grindavik,IS,63.8424,-22.4338,28.40,69,75,23.02,broken clouds
2,2,Ribas Do Rio Pardo,BR,-20.4431,-53.7592,70.03,95,100,6.22,overcast clouds
3,3,Awjilah,LY,29.1081,21.2869,58.64,35,25,10.31,scattered clouds
4,4,Butaritari,KI,3.0707,172.7902,80.01,81,70,12.82,light rain


In [11]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? ")) #this gets a floaing-poimt decimal in int input()
max_temp = float(input("What is the maximum temperature you would like for your trip? ")) #example float(input())

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [12]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
4,4,Butaritari,KI,3.0707,172.7902,80.01,81,70,12.82,light rain
6,6,Hermanus,ZA,-34.4187,19.2345,89.01,50,1,7.00,clear sky
7,7,Cape Town,ZA,-33.9258,18.4232,78.80,53,0,12.66,clear sky
8,8,Trelew,AR,-43.2490,-65.3051,76.08,30,0,13.00,clear sky
13,13,Sibolga,ID,1.7427,98.7792,81.55,73,100,2.93,light rain
14,14,Port Alfred,ZA,-33.5906,26.8910,78.01,80,33,10.00,scattered clouds
15,15,Caruray,PH,10.3094,119.0128,82.40,83,40,6.91,scattered clouds
22,22,Bac Lieu,VN,9.2850,105.7244,78.91,76,92,8.48,overcast clouds
23,23,Natal,BR,-5.7950,-35.2094,75.20,88,20,4.61,few clouds
30,30,Rikitea,PF,-23.1203,-134.9692,78.42,73,0,13.78,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count() # this determine if there are any null values.  186 in all rows so no null 

Unnamed: 0             186
City                   186
Lat                    186
Lng                    186
Max Temp               186
Humidity               186
Cloudiness             186
Wind Speed             186
Weather Description    186
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
#no empty rows but we could use the dropna()

In [14]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
#Country was not called in our csv file WeatherPy_database.csv from step 6 of deliverable 1 so we can't pull into this dataframe

hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Weather Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
4,Butaritari,KI,80.01,light rain,3.0707,172.7902,
6,Hermanus,ZA,89.01,clear sky,-34.4187,19.2345,
7,Cape Town,ZA,78.80,clear sky,-33.9258,18.4232,
8,Trelew,AR,76.08,clear sky,-43.2490,-65.3051,
13,Sibolga,ID,81.55,light rain,1.7427,98.7792,
14,Port Alfred,ZA,78.01,scattered clouds,-33.5906,26.8910,
15,Caruray,PH,82.40,scattered clouds,10.3094,119.0128,
22,Bac Lieu,VN,78.91,overcast clouds,9.2850,105.7244,
23,Natal,BR,75.20,few clouds,-5.7950,-35.2094,
30,Rikitea,PF,78.42,clear sky,-23.1203,-134.9692,


In [15]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]  
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [17]:
hotel_df.count()

City                   206
Country                205
Max Temp               206
Weather Description    206
Lat                    206
Lng                    206
Hotel Name             206
dtype: int64

In [19]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df = hotel_df.dropna()
hotel_df

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
4,Butaritari,KI,80.01,light rain,3.0707,172.7902,Isles Sunset Lodge
6,Hermanus,ZA,89.01,clear sky,-34.4187,19.2345,Misty Waves Boutique Hotel
7,Cape Town,ZA,78.80,clear sky,-33.9258,18.4232,"Radisson Blu Hotel Waterfront, Cape Town"
8,Trelew,AR,76.08,clear sky,-43.2490,-65.3051,Hotel Touring Club
13,Sibolga,ID,81.55,light rain,1.7427,98.7792,Hotel Syariah CN Darussalam
...,...,...,...,...,...,...,...
671,Iquitos,PE,75.20,broken clouds,-3.7481,-73.2472,Cortez Malaga Techy Caroly
673,Bathsheba,BB,78.80,few clouds,13.2167,-59.5167,Atlantis Hotel
674,Hervey Bay,AU,75.99,overcast clouds,-25.2986,152.8535,Shelly Bay Resort
682,Port-Gentil,GA,78.80,light rain,-0.7193,8.7815,Fengshui Residence


In [22]:
# 8a. Create the output File (CSV)
output_data_file = os.path.join('..', 'Vacation_Search', 'WeatherPy_vacation.csv')
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [31]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Weather Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# 11b. Display the figure
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))